# Portfolio Hedging\n

In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.decomposition import PCA
np.random.seed(0)
T=300
a1=np.cumsum(np.random.normal(0,1,T))
a2=a1*0.6+np.cumsum(np.random.normal(0,0.5,T))
a3=a1*0.3+np.cumsum(np.random.normal(0,1,T))*0.2
df = pd.DataFrame({'a1':a1,'a2':a2,'a3':a3})
pca=PCA(n_components=2).fit(df)
print('explained', pca.explained_variance_ratio_)
